In [177]:
#| default_exp components


In [178]:
#| export
import fasthtml.common as fh
from fh_flowbite.core import *
from fasthtml.common import Div, P, Span, FT, HighlightJS
from enum import Enum, auto
from fasthtml.components import Uk_select,Uk_input_tag,Uk_icon
from functools import partial
from itertools import zip_longest
from typing import Union, Tuple, Optional, Sequence
from fastcore.all import *
import copy, re, httpx, os
from pathlib import Path
# from mistletoe.html_renderer import HTMLRenderer
# from mistletoe.span_token import Image
from pathlib import Path
# import mistletoe
# from lxml import html, etree
from fasthtml.components import Uk_input_range
import fasthtml.components as fh_comp

In [179]:
#| export
flowbite_hdrs = (
    Script(src="https://unpkg.com/@tailwindcss/browser@4"),
    Link(
        rel="stylesheet",
        href="https://cdn.jsdelivr.net/npm/flowbite@3.1.2/dist/flowbite.min.css",
    ),
    Link(rel="stylesheet", href="/output.css"),
    Script(
        "if (localStorage.getItem('color-theme') === 'dark' || (!('color-theme' in localStorage) && window.matchMedia('(prefers-color-scheme: dark)').matches)) {\r\n        document.documentElement.classList.add('dark');\r\n    } else {\r\n        document.documentElement.classList.remove('dark')\r\n    }"
    ),
    # Script(src="/tailwind.config.js"),
)

flowbite_ftrs = [
    Script(src="https://cdn.jsdelivr.net/npm/flowbite@3.1.2/dist/flowbite.min.js"),
]

In [180]:
#| export
class TextT(VEnum):
    """
    Text Styles from Flowbite UI
    """
    
    # Text Size
    xs, sm, base, lg, xl = 'text-xs', 'text-sm', 'text-base', 'text-lg', 'text-xl'
    text_2xl, text_3xl, text_4xl = 'text-2xl', 'text-3xl', 'text-4xl'
    text_5xl, text_6xl, text_7xl = 'text-5xl', 'text-6xl', 'text-7xl'
    text_8xl, text_9xl = 'text-8xl', 'text-9xl'
    
    # Text Weight
    thin, extralight, light = 'font-thin', 'font-extralight', 'font-light'
    normal, medium, semibold = 'font-normal', 'font-medium', 'font-semibold'
    bold, extrabold, black = 'font-bold', 'font-extrabold', 'font-black'
    
    # Text Style
    italic = 'italic'
    
    # Text Color
    gray = 'text-gray-500 dark:text-gray-400'
    muted = 'text-gray-500 dark:text-gray-400'
    primary = 'text-primary-600 dark:text-primary-500'
    # TODO: Add secondary color
    secondary = 'text-purple-600 dark:text-purple-500'
    success = 'text-green-500 dark:text-green-400'
    warning = 'text-yellow-500 dark:text-yellow-400'
    error = 'text-red-600 dark:text-red-500'
    info = 'text-blue-500 dark:text-blue-400'
    
    # Text Alignment
    left, right, center = 'text-left', 'text-right', 'text-center'
    justify, start, end = 'text-justify', 'text-start', 'text-end'
    
    # Vertical Alignment
    top, middle, bottom = 'align-top', 'align-middle', 'align-bottom'
    
    # Text Decoration
    underline, line_through = 'underline', 'line-through'
    uppercase, lowercase = 'uppercase', 'lowercase'
    capitalize, normal_case = 'capitalize', 'normal-case'
    
    # Line Height
    leading_none = 'leading-none'
    leading_tight = 'leading-tight'
    leading_snug = 'leading-snug'
    leading_normal = 'leading-normal'
    leading_relaxed = 'leading-relaxed'
    leading_loose = 'leading-loose'
    
    # Letter Spacing
    tracking_tighter = 'tracking-tighter'
    tracking_tight = 'tracking-tight'
    tracking_normal = 'tracking-normal'
    tracking_wide = 'tracking-wide'
    tracking_wider = 'tracking-wider'
    tracking_widest = 'tracking-widest'
    
    # Text Wrapping
    truncate = 'truncate'
    whitespace_normal = 'whitespace-normal'
    whitespace_nowrap = 'whitespace-nowrap'
    whitespace_pre = 'whitespace-pre'
    whitespace_pre_line = 'whitespace-pre-line'
    whitespace_pre_wrap = 'whitespace-pre-wrap'
    
    # Other
    highlight = 'bg-yellow-200 dark:bg-yellow-800 text-black'


class TextPresets(VEnum):
    """
    Common Typography Presets for Flowbite UI
    """
    muted_sm = TextT.muted+TextT.sm
    muted_lg = TextT.muted+TextT.lg
    
    bold_sm = TextT.bold+TextT.sm
    bold_lg = TextT.bold+TextT.lg
    
    primary_bold = stringify((TextT.primary, TextT.bold))
    success_bold = stringify((TextT.success, TextT.bold))
    warning_bold = stringify((TextT.warning, TextT.bold))
    error_bold = stringify((TextT.error, TextT.bold))
    
    md_weight_sm = stringify((TextT.sm, TextT.medium))
    md_weight_muted = stringify((TextT.medium, TextT.muted))

class TextHeading(VEnum):
    """Text heading variants for Flowbite components"""
    h1 = "text-5xl font-extrabold dark:text-white mb-4"
    h2 = "text-4xl font-bold dark:text-white mb-3"
    h3 = "text-3xl font-bold dark:text-white mb-2"
    h4 = "text-2xl font-bold dark:text-white mb-1"
    h5 = "text-xl font-bold dark:text-white mb-0.5"
    h6 = "text-lg font-bold dark:text-white"

def H1(*c:FT|str, # Contents of H1 tag (often text)
       cls:Enum|str|tuple=(), # Classes in addition to H1 styling
       **kwargs # Additional args for H1 tag
       )->FT: # H1(..., cls='uk-h1')
    "H1 with styling and appropriate size"
    return fh.H1(*c, cls=(TextHeading.h1, stringify(cls)), **kwargs)

def H2(*c:FT|str, # Contents of H2 tag (often text)
       cls:Enum|str|tuple=(), # Classes in addition to H2 styling
       **kwargs # Additional args for H2 tag
       )->FT: # H2(..., cls='uk-h2')
    "H2 with styling and appropriate size"
    return fh.H2(*c, cls=(TextHeading.h2, stringify(cls)), **kwargs)

def H3(*c:FT|str, # Contents of H3 tag (often text)
       cls:Enum|str|tuple=(), # Classes in addition to H3 styling
       **kwargs # Additional args for H3 tag
       )->FT: # H3(..., cls='uk-h3')
    "H3 with styling and appropriate size"
    return fh.H3(*c, cls=(TextHeading.h3, stringify(cls)), **kwargs)

def H4(*c:FT|str, # Contents of H4 tag (often text)
       cls:Enum|str|tuple=(), # Classes in addition to H4 styling
       **kwargs # Additional args for H4 tag
       )->FT: # H4(..., cls='uk-h4')
    "H4 with styling and appropriate size"
    return fh.H4(*c, cls=(TextHeading.h4, stringify(cls)), **kwargs)

def H5(*c:FT|str, # Contents of H5 tag (often text)
       cls:Enum|str|tuple=(), # Classes in addition to H5 styling
       **kwargs # Additional args for H5 tag
       )->FT: # H5(..., cls='uk-h5')
    "H5 with styling and appropriate size"
    return fh.H5(*c, cls=(TextHeading.h5, stringify(cls)), **kwargs)

def H6(*c:FT|str, # Contents of H6 tag (often text)
       cls:Enum|str|tuple=(), # Classes in addition to H6 styling
       **kwargs # Additional args for H6 tag
       )->FT: # H6(..., cls='uk-h6')
    "H6 with styling and appropriate size"
    return fh.H6(*c, cls=(TextHeading.h6, stringify(cls)), **kwargs)


In [181]:
#| export
def Meter(*c:FT|str, # Contents of Meter tag
          value:float=None, # Current value
          min:float=None, # Minimum value
          max:float=None, # Maximum value
          cls:Enum|str|tuple=(), # Additional classes
          **kwargs # Additional args for Meter tag
          )->FT:
    "Styled meter element"
    if value is not None: kwargs['value'] = value
    if min is not None: kwargs['min'] = min
    if max is not None: kwargs['max'] = max
    return fh.Meter(*c, cls=('w-full h-2 rounded', stringify(cls)), **kwargs)


In [182]:
#| export
class ButtonColor(VEnum):
    """Button style variants for Flowbite components"""
    # Base styles
    primary = "bg-primary-700 hover:bg-primary-800 dark:bg-primary-600 dark:hover:bg-primary-700  text-white focus:ring-primary-300 dark:focus:ring-primary-800"
    green = "bg-green-700 hover:bg-green-800 dark:bg-green-600 dark:hover:bg-green-700 dark:focus:ring-green-800 text-white focus:ring-green-300"
    blue = "bg-blue-700 hover:bg-blue-800 dark:bg-blue-600 dark:hover:bg-blue-700 dark:focus:ring-blue-800 text-white focus:ring-blue-300"
    red = "bg-red-700 hover:bg-red-800 dark:bg-red-600 dark:hover:bg-red-700 dark:focus:ring-red-900 text-white focus:ring-red-300"
    yellow = "bg-yellow-400 hover:bg-yellow-500 dark:focus:ring-yellow-900 text-white focus:ring-yellow-300"
    purple = "bg-purple-700 hover:bg-purple-800 dark:bg-purple-600 dark:hover:bg-purple-700 dark:focus:ring-purple-900 text-white focus:ring-purple-300"

class Round(VEnum):
    """Button shape variants"""
    default = "rounded-lg"
    full = "rounded-full"
    none = "rounded-none"
    sm = "rounded-sm"
    md = "rounded-md"
    lg = "rounded-lg"
    xl = "rounded-xl"
    xxl = "rounded-2xl"
    xxxl = "rounded-3xl"
    xxxxl = "rounded-4xl"

class ButtonSize(VEnum):
    """Button size variants"""
    xs = "px-3 py-2 text-xs font-medium"
    sm = "px-3 py-2 text-sm font-medium"
    base = "px-5 py-2.5 text-sm font-medium"
    lg = "px-5 py-3 text-base font-medium"
    xl = "px-6 py-3.5 text-base font-medium"
    
class ButtonOutline(VEnum):
    """Button outline variants"""
    default = "border-blue-700 hover:bg-blue-800 dark:border-blue-500 dark:hover:bg-blue-500 text-blue-700 hover:text-white dark:text-blue-500 dark:hover:text-white focus:ring-blue-300 dark:focus:ring-blue-800"
    dark = "border-gray-800 hover:bg-gray-900 dark:border-gray-600 dark:hover:bg-gray-600 text-gray-900 hover:text-white dark:text-gray-400 dark:hover:text-white focus:ring-gray-300 dark:focus:ring-gray-800"
    green = "border-green-700 hover:bg-green-800 dark:border-green-500 dark:hover:bg-green-600 text-green-700 hover:text-white dark:text-green-500 dark:hover:text-white focus:ring-green-300 dark:focus:ring-green-800"
    red = "border-red-700 hover:bg-red-800 dark:border-red-500 dark:hover:bg-red-600 text-red-700 hover:text-white dark:text-red-500 dark:hover:text-white focus:ring-red-300 dark:focus:ring-red-900"
    yellow = "border-yellow-400 hover:bg-yellow-500 dark:border-yellow-300 dark:hover:bg-yellow-400 text-yellow-400 hover:text-white dark:text-yellow-300 dark:hover:text-white focus:ring-yellow-300 dark:focus:ring-yellow-900"
    purple = "border-purple-700 hover:bg-purple-800 dark:border-purple-400 dark:hover:bg-purple-500 text-purple-700 hover:text-white dark:text-purple-400 dark:hover:text-white focus:ring-purple-300 dark:focus:ring-purple-900"

class ButtonGradientOutline(VEnum):
    """Gradient outline button variants"""
    purple_blue = "group-hover:from-purple-600 group-hover:to-blue-500 hover:text-white dark:text-white focus:ring-blue-300 dark:focus:ring-blue-800 group bg-gradient-to-br from-purple-600 to-blue-500 relative inline-flex items-center justify-center p-0.5 overflow-hidden text-sm font-medium text-gray-900 focus:ring-4 focus:outline-none"
    cyan_blue = "group-hover:from-cyan-500 group-hover:to-blue-500 hover:text-white dark:text-white focus:ring-cyan-200 dark:focus:ring-cyan-800 group bg-gradient-to-br from-cyan-500 to-blue-500 relative inline-flex items-center justify-center p-0.5 overflow-hidden text-sm font-medium text-gray-900 focus:ring-4 focus:outline-none"
    green_blue = "group-hover:from-green-400 group-hover:to-blue-600 hover:text-white dark:text-white focus:ring-green-200 dark:focus:ring-green-800 group bg-gradient-to-br from-green-400 to-blue-600 relative inline-flex items-center justify-center p-0.5 overflow-hidden text-sm font-medium text-gray-900 focus:ring-4 focus:outline-none"
    purple_pink = "group-hover:from-purple-500 group-hover:to-pink-500 hover:text-white dark:text-white focus:ring-purple-200 dark:focus:ring-purple-800 group bg-gradient-to-br from-purple-500 to-pink-500 relative inline-flex items-center justify-center p-0.5 overflow-hidden text-sm font-medium text-gray-900 focus:ring-4 focus:outline-none"
    pink_orange = "group-hover:from-pink-500 group-hover:to-orange-400 hover:text-white dark:text-white focus:ring-pink-200 dark:focus:ring-pink-800 group bg-gradient-to-br from-pink-500 to-orange-400 relative inline-flex items-center justify-center p-0.5 overflow-hidden text-sm font-medium text-gray-900 focus:ring-4 focus:outline-none"
    teal_lime = "group-hover:from-teal-300 group-hover:to-lime-300 dark:text-white dark:hover:text-gray-900 focus:ring-lime-200 dark:focus:ring-lime-800 group bg-gradient-to-br from-teal-300 to-lime-300 relative inline-flex items-center justify-center p-0.5 overflow-hidden text-sm font-medium text-gray-900 focus:ring-4 focus:outline-none"
    red_yellow = "group-hover:from-red-200 group-hover:via-red-300 group-hover:to-yellow-200 dark:text-white dark:hover:text-gray-900 focus:ring-red-100 dark:focus:ring-red-400 group bg-gradient-to-br from-red-200 via-red-300 to-yellow-200 relative inline-flex items-center justify-center p-0.5 overflow-hidden text-sm font-medium text-gray-900 focus:ring-4 focus:outline-none"

class ButtonT(VEnum):
    """Button type variants"""
    primary = ButtonColor.primary.value
    secondary = "bg-white border border-gray-300 hover:bg-gray-100 dark:bg-gray-800 dark:text-white dark:border-gray-600 dark:hover:bg-gray-700 dark:hover:border-gray-600 text-gray-900 focus:ring-gray-200 dark:focus:ring-gray-700"
    alternative = "bg-white border border-gray-200 hover:bg-gray-100 hover:text-primary-700 dark:bg-gray-800 dark:border-gray-600 dark:hover:text-white dark:hover:bg-gray-700 text-gray-600 dark:text-gray-600 focus:ring-gray-200 dark:focus:ring-gray-700"
    ghost = "text-white hover:bg-gray-100 hover:text-primary-700 dark:hover:text-white dark:hover:bg-gray-700 focus:ring-gray-300 dark:focus:ring-gray-700 cursor-pointer"
    link = "text-white cursor-pointer"
    success = ButtonColor.green.value   
    warning = ButtonColor.yellow.value
    error = ButtonColor.red.value
    info = ButtonColor.blue.value
    

In [183]:
#| export
def Button(*c: Union[str, FT],cls: Union[str, Enum]=ButtonT.primary, shape: Union[str, Enum]=Round.default, submit=True, **kwargs) -> FT: 
    base_class = "font-medium text-sm px-5 py-2.5 focus:outline-none focus:ring-4 text-center inline-flex items-center"
    if 'type' not in kwargs: kwargs['type'] = 'submit' if submit else 'button'
    return fh.Button(*c, cls=(base_class, stringify(shape), stringify(cls)), **kwargs)


AttributeError: type object 'ButtonColor' has no attribute 'default'

In [159]:
#| export
class FlexT(VEnum):
    """Flexbox utility types"""
    def _generate_next_value_(name, start, count, last_values): 
        return str2cls('flex', name)
    
    # Display
    flex = "flex"
    inline = "inline-flex"
    
    # Direction
    row = auto()  # Will generate "flex-row"
    row_reverse = auto()  # Will generate "flex-row-reverse"
    col = auto()  # Will generate "flex-col"
    col_reverse = auto()  # Will generate "flex-col-reverse"
    
    # Justify content
    justify_start = "justify-start"
    justify_end = "justify-end"
    justify_center = "justify-center"
    justify_between = "justify-between"
    justify_around = "justify-around"
    justify_evenly = "justify-evenly"
    
    # Align items
    items_start = "items-start"
    items_end = "items-end"
    items_center = "items-center"
    items_baseline = "items-baseline"
    items_stretch = "items-stretch"
    
    # Wrap
    wrap = auto()  # Will generate "flex-wrap"
    wrap_reverse = auto()  # Will generate "flex-wrap-reverse"
    nowrap = auto()  # Will generate "flex-nowrap" 

In [160]:
#| export
class FlexT(VEnum):
    """Flexbox utility types"""
    def _generate_next_value_(name, start, count, last_values): 
        return str2cls('flex', name)
    
    # Display
    flex = "flex"
    inline = "inline-flex"
    
    # Direction
    row = auto()  # Will generate "flex-row"
    row_reverse = auto()  # Will generate "flex-row-reverse"
    col = auto()  # Will generate "flex-col"
    col_reverse = auto()  # Will generate "flex-col-reverse"
    
    # Justify content
    justify_start = "justify-start"
    justify_end = "justify-end"
    justify_center = "justify-center"
    justify_between = "justify-between"
    justify_around = "justify-around"
    justify_evenly = "justify-evenly"
    
    # Align items
    items_start = "items-start"
    items_end = "items-end"
    items_center = "items-center"
    items_baseline = "items-baseline"
    items_stretch = "items-stretch"
    
    # Wrap
    wrap = auto()  # Will generate "flex-wrap"
    wrap_reverse = auto()  # Will generate "flex-wrap-reverse"
    nowrap = auto()  # Will generate "flex-nowrap" 

In [161]:
#| hide
import nbdev; nbdev.nbdev_export()